# BRAF Mutation Trans Effect on Transcriptomics

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")

import cptac
import cptac.utils as al

brain = cptac.Gbm()

### Select Gene

In [2]:
gene = "BRAF"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
omics = "transcriptomics"
#omics = "phosphoproteomics"
#omics = "acetylproteomics"

### Track all significant comparisons in Dataframe

In [4]:
all_significant_comparisons = pd.DataFrame(columns=['Cancer_Type', 'Gene', 'Comparison','Interacting_Protein','P_Value'])

In [5]:
def add_to_all_significant_comparisons(df, cancer, interacting, all_sig_comp):
    expanded = df
    expanded['Gene'] = gene
    expanded['Cancer_Type'] = cancer
    expanded['Interacting_Protein'] = interacting
    
    updated_all_comparisons = pd.concat([all_sig_comp, expanded], sort=False)
    
    return updated_all_comparisons

# Interacting Proteins: Transcriptomics

### Generate interacting protein list

Make a call to cptac.utils to get the interacting proteins method, which interacts with the uniprot and string databases to generate a list of known interacting partners with the given gene.

In [6]:
# Use get interacting proteins method to generate list of interacting proteins
interacting_proteins = al.get_interacting_proteins(gene)

# Show interacting protein list
print("Interacting Proteins:")
for interacting_protein in interacting_proteins:
    print(interacting_protein)

Interacting Proteins:
YWHAH
MAPK1
RAP1B
KRAS
MAP2K2
MAPK3
YWHAE
IQGAP1
BRAF
MAP2K1
YWHAG
PRKACA
SFN
NRAS
RAP1A
YWHAB
CNKSR1
SPRY2
YWHAQ
YWHAZ
KSR1
KSR2
HRAS
PRKCA
KIAA1549
LAMTOR3
AKT1
RAF1
MRAS
NEDD4L
Nedd4


### Test for significant comparisons in any of interacting proteins

In [7]:
# Create dataframe in order to do comparisons with wrap_ttest
protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = brain.reduce_multiindex(protdf, flatten=True)
protdf = protdf[protdf.Sample_Status == 'Tumor'] # drop Normal samples

# Create the binary valued column needed to do the comparison
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

# Format the dataframe correctly
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


# Make list of columns to be compared using t-tests
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

# Call wrap_ttest, pass in formatted dataframe
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

# Print results, if anything significant was found
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", True, all_significant_comparisons)
        

Doing t-test comparisons

No significant comparisons.


# All Proteins: Transcriptomics 

In [8]:
try:
    print("\nGene: ", gene)

    # Use all proteins

    # Create dataframe in order to do comparisons with wrap_ttest
    protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
    protdf = brain.reduce_multiindex(protdf, flatten=True) # flatten for specific column names
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    # Create binary column 
    protdf['Label'] = np.where(
                protdf[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

    # Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    # Make list of columns to be compared using t-tests
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    # Call wrap_ttest, pass in formatted dataframe
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    # Print results, if anything significant was found
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")
            
            all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", False, all_significant_comparisons)


except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  BRAF


Doing t-test comparisons

                                      Comparison       P_Value
0        ESR2_transcriptomics_ENSG00000140009.17  1.106272e-19
1        GNA14_transcriptomics_ENSG00000156049.6  2.265766e-16
2          SHH_transcriptomics_ENSG00000164690.6  6.291187e-16
3   AC110056.1_transcriptomics_ENSG00000211499.1  4.712997e-14
4        MRPS6_transcriptomics_ENSG00000243927.4  2.407117e-13
..                                           ...           ...
83      TSHZ2_transcriptomics_ENSG00000182463.14  7.277464e-07
84     RASSF8_transcriptomics_ENSG00000123094.14  7.341287e-07
85      COPS2_transcriptomics_ENSG00000166200.13  7.515065e-07
86       PAPPA_transcriptomics_ENSG00000182752.9  7.834591e-07
87       MREG_transcriptomics_ENSG00000118242.14  8.072223e-07

[88 rows x 2 columns]





# Print all significant comparisons

In [9]:
if len(all_significant_comparisons) > 0:
    display(all_significant_comparisons)
    
else:
    print('No Significant Comparisons!')

,Cancer_Type,Gene,Comparison,Interacting_Protein,P_Value
0,Gbm,BRAF,ESR2_transcriptomics_ENSG00000140009.17,False,1.106272e-19
1,Gbm,BRAF,GNA14_transcriptomics_ENSG00000156049.6,False,2.265766e-16
2,Gbm,BRAF,SHH_transcriptomics_ENSG00000164690.6,False,6.291187e-16
3,Gbm,BRAF,AC110056.1_transcriptomics_ENSG00000211499.1,False,4.712997e-14
4,Gbm,BRAF,MRPS6_transcriptomics_ENSG00000243927.4,False,2.407117e-13
...,...,...,...,...,...
83,Gbm,BRAF,TSHZ2_transcriptomics_ENSG00000182463.14,False,7.277464e-07
84,Gbm,BRAF,RASSF8_transcriptomics_ENSG00000123094.14,False,7.341287e-07
85,Gbm,BRAF,COPS2_transcriptomics_ENSG00000166200.13,False,7.515065e-07
86,Gbm,BRAF,PAPPA_transcriptomics_ENSG00000182752.9,False,7.834591e-07


In [19]:
all_significant_comparisons['Comparison'].replace(to_replace = '_transcriptomics.*$', value = '', 
                        inplace = True, regex = True) # shorten column names
all_significant_comparisons['Comparison'].unique()

array(['ESR2', 'GNA14', 'SHH', 'AC110056.1', 'MRPS6', 'RNU6-1303P',
       'CNTNAP3', 'SCG2', 'PLAG1', 'WWP1', 'DCBLD2', 'RP11-203I2.1',
       'COL13A1', 'PVRL3', 'SLC5A3', 'NPAS2', 'TXNDC15', 'RP11-203M5.6',
       'AC027238.1', 'AP1S3', 'RP11-35O15.1', 'IDS', 'COL8A1', 'NHSL2',
       'BMS1P17', 'PROS1', 'RP11-365H8.2', 'CAMK2D', 'RP11-1081M5.2',
       'IGKV1OR-3', 'KCNK15', 'ADH1C', 'APLP2', 'STEAP2', 'RP11-587D21.4',
       'IL6ST', 'AC092168.2', 'RP3-512B11.3', 'AC022819.2', 'AC004969.1',
       'RP11-262D11.2', 'RP11-140I16.3', 'GRIA1', 'HORMAD2-AS1', 'CLASP1',
       'APOA1', 'KLRF2', 'TRH', 'SEMA3A', 'SLC20A1', 'JPH1', 'DPEP3',
       'RP5-1077H22.1', 'PBX2P1', 'ENDOD1', 'DIRAS3', 'POSTN', 'CHCHD2P7',
       'B3GALT1', 'ITFG1', 'ARSI', 'SVIP', 'PTPRH', 'RN7SKP258', 'CGB7',
       'CPNE3', 'RNU6-26P', 'CNTNAP3B', 'RRM2B', 'RP11-80H8.4', 'ANO1',
       'RGS17', 'KIAA1024', 'RP11-460B17.2', 'ITGA2', 'ADAMTSL1',
       'HOMER1', 'LINC00284', 'AC108057.1', 'CTB-161J7.1', 'TMEM108-

### Write Significant Comparisons (if any) to Shared CSV file

In [11]:

existing_results = pd.read_csv(gene+'_Trans_Results.csv')

updated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)

updated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)